## Modelado de lenguaje con Transformers

Vamos a introducir un texto en un Transformer y examinar cómo calcula las probabilidades de la siguiente palabra/token.

Primero cargaremos el tokenizador `distilgpt2` como hicimos anteriormente.


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('distilgpt2')

 Nos interesará predecir el siguiente token subpalabra. ¿Cuántos tokens subpalabra posibles hay?

In [ ]:
len(tokenizer.vocab) # o deberiamos usar len(tokenizer)

Al tokenizar, usaremos el tokenizador con el parámetro `return_tensors='pt'`. Esto coloca los datos en el formato de un tensor de [PyTorch](https://pytorch.org), que se utiliza como entrada para un modelo Transformer. PyTorch es una biblioteca comúnmente utilizada para aprendizaje profundo y HuggingFace se basa en ella. No usaremos PyTorch directamente.

Vamos a tokenizar: `"A horse! a horse! my kingdom for a"`


In [ ]:
tokenized = tokenizer('A horse! a horse! my kingdom for a', return_tensors='pt')
tokenized

Observa que ha sido tokenizado en 10 tokens.

In [ ]:
len(tokenized['input_ids'][0])

Ahora necesitamos cargar el modelo Transformer completo. Necesitamos usar el mismo que coincide con nuestro tokenizador (`distilgpt2`). Los tokenizadores y modelos deben coincidir.

Lo cargaremos usando `AutoModelForCausalLM`. CausalLM es modelado de lenguaje causal, o predicción del siguiente token. También puedes cargar modelos para otros propósitos como clasificación de documentos.

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained('distilgpt2')

Ahora pasemos el texto tokenizado al modelo Transformer. Podríamos hacer esto con `model(input_ids=tokenized['input_ids'], attention_mask=tokenized['attention_mask'])` pero una forma abreviada y ordenada es:

In [ ]:
output = model(**tokenized)

En el modelado de lenguaje causal, lo que nos interesa son las predicciones del siguiente token. Esto se captura en los `logits`, que son las puntuaciones para cada uno de los tokens posibles.

In [ ]:
output.logits

Esto es un tensor de PyTorch que es una cuadrícula de números. En este caso, es una cuadrícula 3D. Puedes ver las dimensiones de la misma usando `.shape` como se muestra a continuación:

In [ ]:
output.logits.shape

¿De dónde vienen los diferentes números?

Bueno, solo introducimos una secuencia de diez palabras, lo que explica el `[1, 10,...]`. El `50257` es el tamaño del vocabulario del tokenizador:

In [ ]:
len(tokenizer)

Eso significa que podemos obtener la puntuación que el Transformer ha dado al token `horse` después del último token en la secuencia. Primero, ¿cuál es el índice del token para horse? Recuerda que como es una nueva palabra, hay el carácter especial de `Ġ`.

In [ ]:
tokenizer.vocab['Ġhorse']

Luego, para obtener la puntuación de la primera secuencia (0), después del último token (-1) y para el token `horse` (8223), lo accederíamos con:

In [ ]:
output.logits[0,-1,8223]

Los logits no son probabilidades agradables por lo que son difíciles de interpretar. Tendremos que hacer un poco de trabajo para hacerlos interpretables.

Vamos a sacar todas las puntuaciones para predicciones de tokens después de nuestra entrada (usando el índice de -1 para obtener los logits finales).

In [ ]:
next_token_scores = output.logits[0,-1,:].tolist()
len(next_token_scores)

Como ya vimos, no son fáciles de interpretar.

In [ ]:
next_token_scores[:5]

Así que usaremos una función softmax. Toma una lista de números, aplica la ecuación a continuación a ellos (usando muchos exponenciales) y devuelve un vector donde todos los valores están entre 0 y 1 y todos suman 1.

$ softmax(z) = \frac{e^{z_{i}}}{\sum_{j=1}^K e^{z_{j}}} \ \ \ for\ i=1,2,\dots,K $

Hay una [función](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.softmax.html) en el útil [paquete scipy](https://scipy.org/) que hace esto por nosotros.

In [ ]:
from scipy.special import softmax

Aplica la función `softmax` a `next_token_scores` y muestra los primeros cinco valores. Deberías ver que están entre 0 y 1 y son bastante pequeños.

In [ ]:
# Tu codigo

Las probabilidades también deben sumar 1 (o muy cerca debido a pequeñas diferencias numéricas). Comprueba si este es el caso usando la función `sum`.

In [ ]:
# Tu codigo


Veamos cuál es la probabilidad de horse ahora (id de token = 8223)

In [ ]:
# Tu codigo

Deberías encontrar que tiene una probabilidad de aproximadamente `0.006`.

Si no supiéramos que 8223 es horse, podríamos decodificarlo con el tokenizador.

In [ ]:
tokenizer.decode(8223)

Ahora, la tarea final es recorrer `next_token_probs` y encontrar cuál tiene la mayor probabilidad y descubrir el el token correspondiente usando `tokenizer.decode`.

In [ ]:
# Tu codigo

Deberías encontrar que `' long'` tiene la mayor probabilidad (`≈ 0.3427`)


### Ejercicios

**Exploración básica del tokenizador**
1. Explora el tokenizador: Utiliza el tokenizador distilgpt2 para explorar cómo se tokenizan diferentes frases. Por ejemplo, intenta tokenizar las frases "Hello, world!" y "¡Hola, mundo!". Compara los resultados y comenta las diferencias.
2. Explora el tamaño del vocabulario del tokenizador distilgpt2. ¿Cuál es el tamaño total del vocabulario? ¿Qué puedes deducir sobre la diversidad de lenguaje que el modelo puede entender basado en el tamaño de su vocabulario?

**Predicción de palabras**

3. Utiliza el modelo distilgpt2 para predecir la siguiente palabra después de la frase "The quick brown fox". Examina los logits retornados y utiliza la función softmax para convertir estos logits en probabilidades. ¿Cuál es la palabra más probable según el modelo?
4. Repite la predicción con varias frases de tu elección que tengan estructuras gramaticales diferentes. ¿Cómo afecta la estructura de la frase a la predicción del modelo?

**Análisis detallado de logits**

5. Interpretación de logits: Utiliza una frase de entrada y obtén los logits para la última palabra. Calcula la probabilidad de las 5 palabras más probables que podrían seguir y decodifícalas para ver qué palabras son.
6. Visualiza las probabilidades de las 10 palabras más probables que podrían seguir después de una entrada dada usando un gráfico de barras. Esto te ayudará a tener una mejor representación visual de cómo el modelo evalúa las posibles continuaciones.

**Experimentación con diferentes modelos**

7. Uso de diferentes modelos: Repite los ejercicios de predicción usando otro modelo de la biblioteca Hugging Face, como gpt-2 o bert-base-uncased. Compara los resultados con los obtenidos usando distilgpt2. ¿Notas alguna mejora o diferencia en las predicciones
8. Utiliza frases con contexto ambiguo y claro, y observa cómo el modelo maneja la ambigüedad en comparación con frases claras. Por ejemplo, compara las predicciones para "bank" en el contexto de un río contra el contexto de una institución financiera.


In [ ]:
## Tus respuestas